In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


In [2]:
# Load the file into pandas
jsonl = '/usr/local/tmp/out.jsonl'
df_orig = ''
with open(jsonl,"r") as json_file:
    data=json_file.read()
    df_orig = pd.read_json(data, lines=True)

df_orig.head()

,audience,categories,concepts,gender,id,max_age,min_age,setting,text
0,3.0,"[1040, 1030]",[20016],0.0,aa100180,64.0,13.0,0.0,polyhydramnios topic overview polyhydramnios...
1,3.0,[1030],"[20058, 20044]",0.0,aa100355,64.0,13.0,0.0,premature delivery in multiple pregnancy topi...
2,3.0,"[1018, 1030]",None,0.0,aa100367,64.0,13.0,0.0,multiple pregnancy: genetic disorders and birt...
3,3.0,[1021],"[21433, 22373]",2.0,aa10253,110.0,2.0,0.0,ambulatory electrocardiogram test overview a...
4,NaN,NaN,NaN,NaN,aa102542,NaN,NaN,NaN,hair growth most scalp hair grows steadily. a...


In [5]:
df_orig['concepts'].value_counts(normalize=True)

TypeError: unhashable type: 'list'

In [6]:
# Get all of the unindexed documents
df_unindexed = df_orig[pd.isnull(df_orig['concepts'])]

In [7]:
df_orig = df_orig[pd.notnull(df_orig['concepts'])]
df_orig['audience'] = df_orig['audience'].astype(int)
df_orig['max_age'] = df_orig['max_age'].astype(int)
df_orig['min_age'] = df_orig['min_age'].astype(int)
df_orig['gender'] = df_orig['gender'].astype(int)
df_orig['setting'] = df_orig['gender'].astype(int)

df_orig.head()

,audience,categories,concepts,gender,id,max_age,min_age,setting,text
0,3,"[1040, 1030]",[20016],0,aa100180,64,13,0,polyhydramnios topic overview polyhydramnios...
1,3,[1030],"[20058, 20044]",0,aa100355,64,13,0,premature delivery in multiple pregnancy topi...
3,3,[1021],"[21433, 22373]",2,aa10253,110,2,2,ambulatory electrocardiogram test overview a...
7,3,"[1018, 1030, 1039]","[21474, 21435, 4124]",0,aa103080,64,13,0,pregnancy: should i have amniocentesis?
9,3,"[1008, 1051]","[22752, 4280]",2,aa104339spec,110,19,2,bodywork and manual therapy topic overview b...


In [8]:
# Build our binary map array for the whole collection since we can't
# be certain that we don't use a label in the test set that isn't
# present in the train set
from sklearn import preprocessing
lb = preprocessing.MultiLabelBinarizer()
Y_all = lb.fit_transform(df_orig['concepts'])
lb

MultiLabelBinarizer(classes=None, sparse_output=False)

In [9]:
ids = df_orig['id']

In [10]:
# Split into test/train
X_train, X_test, Y_train, Y_test, id_train, id_test = train_test_split(df_orig, Y_all, ids, test_size=0.2, random_state=1)

print("X Training set: ", len(X_train))
print("Y Training set: ", len(Y_train))
print("X Testing set: ", len(X_test))
print("Y Testing set: ", len(Y_test))
print("IDs Test: ", len(id_train))
print("IDs Train: ", len(id_test))

X Training set:  7060
Y Training set:  7060
X Testing set:  1766
Y Testing set:  1766
IDs Test:  7060
IDs Train:  1766


from sklearn import preprocessing
lb = preprocessing.MultiLabelBinarizer()
Y_train = lb.fit_transform(X_train['concepts'])
print("Training Y: ", len(Y_train))
Y_test = lb.fit_transform(X_test['concepts'])
print("Testing Y: ", len(Y_test))
print("Size of Y array: ",len(Y_test[0]))
bm_all = lb.fit_transform(df_orig['concepts'])
print("Full: ", len(bm_all))
print("Size of All array: ",len(bm_all[0]))

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

classifier = Pipeline([
('vectorizer', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', OneVsRestClassifier(LinearSVC()))])

classifier.fit(X_train['text'], Y_train)

/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 15 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 34 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 41 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 107 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 117 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/l

/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 933 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 970 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 981 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 987 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1013 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1035 is present in all training examples.
  str(classes[c]))
/Users/akimball/anac

/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1710 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1721 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1732 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1735 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1765 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1785 is present in all training examples.
  str(classes[c]))
/Users/akimball/

/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2153 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2256 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2266 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2276 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2323 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2327 is present in all training examples.
  str(classes[c]))
/Users/akimball/

/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2689 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2704 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2707 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2721 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2726 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2728 is present in all training examples.
  str(classes[c]))
/Users/akimball/

/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3030 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3041 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3043 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3050 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3057 is present in all training examples.
  str(classes[c]))
/Users/akimball/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3062 is present in all training examples.
  str(classes[c]))
/Users/akimball/

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...lti_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1))])

In [12]:
Y_predicted = classifier.predict(X_test['text'])
print("Test predictions: ",len(Y_predicted))

Test predictions:  1766


import numpy
for count, prediction in enumerate(Y_predicted):
    answer = Y_test[count]
    print("Prediction length: ",len(prediction))
    print("Answer length: ",len(answer))

    print("--")
    if np.array_equal(prediction, answer):
        print("!")

In [13]:
from sklearn.metrics import accuracy_score
print(len(Y_test))
print(len(Y_predicted))
print("Accuracy Score: ",accuracy_score(Y_predicted, Y_test))


1766
1766
Accuracy Score:  0.236126840317


for id,concept in zip()

In [14]:
inversed = lb.inverse_transform(Y_predicted)
actual_inverse = lb.inverse_transform(Y_test)

In [15]:
for id, inverse, id, actual in zip(Y_predicted, inversed, id_test, actual_inverse):
    print('%r => %s | %s' % (id, inverse, actual))

#for doc, category in zip(docs_new, predicted):
#    print('%r => %s' % (doc, category))

'sig49735' => () | (20222, 20233)
'abq7361' => () | (22527,)
'aa4617spec' => () | (20534,)
'acc4343' => () | (23229,)
'abp9705' => (4650,) | (22699,)
'hw120667' => (20502,) | (4536, 20502)
'hw190019' => (20590,) | (20590,)
'hw183346' => () | (20607,)
'abj6977' => () | (22645,)
'abp7096' => () | (4176,)
'tb1311' => (20131,) | (20131, 21762)
'abn2238' => () | (5176, 20418, 20780)
'uh4967' => () | (20878,)
'abj9909' => () | (21003,)
'abn0740' => (20272,) | (20272,)
'uh3215' => (20193,) | (20193,)
'abr3526' => (21316,) | (4330, 20705, 21316)
'ut2392' => (4555,) | (4555,)
'bz1061' => () | (20876,)
'tw12221' => (20446, 20447) | (21193,)
'zp2624' => (20236,) | (20236, 21418)
'not254494' => (20857,) | (20851,)
'ug3843' => () | (20135, 21830)
'ad1506' => (4288,) | (4288, 20927)
'aa15658' => (20447,) | (4551, 20446, 20447)
'ux1017' => () | (20982,)
'tn8456' => () | (20046,)
'hw222923' => (20145,) | (20145,)
'abk6940' => () | (4538, 22328)
'abo4389' => (4126,) | (4126, 4599, 22545)
'zp4185' => ()

'uf8208' => () | (20766,)
'ug3907' => () | (5014, 20927, 21925)
'ncicdr0000258025' => () | (21380,)
'abp7900' => () | (4748, 20795, 20796)
'abp7330' => () | (5080, 5081, 20123, 21948, 22137, 22139, 22140)
'tx4083abc' => (20706,) | (20251, 20371, 20706)
'zx3052' => (20799,) | (20799,)
'zc2667' => () | (21939, 22877)
'abk1346' => (4374,) | (21290,)
'abp9955' => () | (22320,)
'uh2749' => () | (21296,)
'aa79756' => () | (21601,)
'abo3062' => () | (23674,)
'abr8922' => () | (20048,)
'aa71858' => (21095,) | (21095, 22306)
'zy1641' => () | (21959,)
'ue4923' => () | (20999,)
'ug2668' => () | (20453,)
'hw164664' => (20511,) | (20511, 20512)
'chbey' => () | (5192, 20934)
'ta6433' => (4315,) | (4315, 4581)
'hw56184' => () | (23026,)
'av2743' => (20889,) | (20889,)
'te2562' => () | (20079, 20100, 21197, 21691)
'bo1717csp' => () | (21966,)
'ug5922' => (20151,) | (20151,)
'ug5448' => () | (20296,)
'hw190690' => (20938,) | (20938,)
'zp4431' => (20126,) | (5157, 20126)
'hw20579' => (21482,) | (20055, 

In [81]:
def fix_hwcvs(ids):
    out = set()
    for i,a in enumerate(ids):
        str_a = a.astype(str)
        if len(str_a) == 4:
            str_a = str("0"+str(a))
        hwcv = 'HWCV_'+str_a
        out.add(hwcv)
    return out
            
            

In [89]:
with open('/usr/local/tmp/results1.csv', 'w') as f: 
    for id, inverse, id, actual in zip(Y_predicted, inversed, id_test, actual_inverse):
        predicted = fix_hwcvs(inverse)
        actual = fix_hwcvs(actual)
        x = id+','+str(predicted)+','+str(actual)
        if predicted == actual:
            x = x+',exact'
        elif predicted.issubset(actual):
            x = x+',partial'
        elif predicted.
        f.write(str(x)+"\n")
